## November 16th Work - 4500 Light Curves from TESS

In [1]:
!pip install lightkurve 

In [2]:
import lightkurve as lk
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from astropy.io import ascii

# If you want your figures to look nicer and larger
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["font.size"] = 20

In [3]:
data = ascii.read("EB_TESScat.ascii")

In [4]:
print(data)

   TIC       period  
---------- ----------
     91961  5.2826248
    101462  4.1739815
    120016  0.3263708
    627436  0.5789058
    737546  3.0677452
    927579  6.0705574
   1003831   1.651149
   1045298  1.4634139
   1103432  3.7260484
   1129033  1.3600996
       ...        ...
1102093577  0.2678155
1400770435 13.7432308
1551345500 47.3858687
1713419350  0.1296923
1883519478  7.6407461
1957912171  0.1185566
1981434198  0.3062443
1981621672  0.2783084
1992266045  6.6177363
2003333263   0.220418
2046417955  0.2358838
Length = 4583 rows


In [5]:
TESS_ID = data['TIC']
Periods = data['period']
Search_Indexes = np.where((Periods>8) & (Periods<12))


In [6]:
print(len(Search_Indexes[0]))

303


In [ ]:
for p, _id in zip(Periods[Search_Indexes], TESS_ID[Search_Indexes]):
    #print(p, _id), and now we have a new table with the period and ID's
    try:
        
        search_result = lk.search_lightcurve(f'TIC {_id}',author="SPOC",exptime=120) #We are querying the TESS catalog so what the telescope observed; we wanted sectors with ID TIC, the author is the process in how the photometry is being measured & SPOC - an automatic way, exptime = exposure time in seconds.
        #print(search_result.mission, type(search_result))

        #mission = search_result['mission']
        #exposure = search_result['exptime']
        #Above is the typical way for calling columns in a data table, but this is a special case where the search result is an object, while the data is a table.

        mission = search_result.mission #Mission was the sector, and a sector is when TESS returns to the same part of the sky and repeats measurements. 
        exposure = search_result.exptime #Exptime was exposure time again in seconds
        #print(mission, exposure)


        list_lc = search_result.download_all() #Downloaded all the available light curves from the available sectors in a list
        
        flux, time = [], [] #Empty lists with flux (or brightness) & time
        for lc in list_lc: #We are going through each light curve and calling it list_lc
            lc = lc.remove_nans() #Removed "empty" or "unavailable" results
            lc = lc.normalize() # normalize light curve - we are saying take all the sectors and make sure they are on the same scale of brightness (electronics can change background noise but for light)
            t, f = lc['time'], lc['flux'] #We are going through each light curve and extracting the time and flux
            flux.append(f.unmasked.value) #Give me all the fluxes that have values (so nans didn't remove all empty data)
            time.append(t.value) #We are appending the time to the empty time list

        master_time = np.concatenate(time) #Stitches them all together
        master_flux = np.concatenate(flux) #Stitches them all together 

        plt.figure(figsize = (15,5))
        plt.scatter(master_time%p, master_flux) #For just light curves do plt.scatter(master_time, master_flux)
        plt.xlabel("Phase[days]")
        plt.ylabel("Flux")
        plt.title(f"Phase Folded Light Curve of TIC {_id}, #Sector(s):{len(mission)}")
        fig_list = plt.savefig(f"TESSPeriodCurves/fig_{_id}.png", bbox_inches='tight', format='png') #I want to make this process automated, meaning it will move files into a labeled folder, but through code b/c manually takes forever). 
        #shutil.move(fig_list, TESSPeriodCurves)
        
    except:
        continue
    

/tmp/ipykernel_304/3769448854.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize = (15,5))


In [ ]:
#Friday: TESS article reading! How many sectores, time for light exposure, investigate light curves. Also modify code to have the observed light curve with the period that spliced together different sectors.
#plt.savefig(f"fig_{_id}.pdf", bbox_inches='tight', format='pdf')

In [8]:
#Goals:
#Part 1 - Organize Files (how do I move them into their own directory?)
#Part 2 - Interpret Files
#Part 3 - Getting them to display on the screen in a nice fashion (should probably take this piece by piece)
#I was able to download over 200 different eliptic binary systems.